In [1]:
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
import ast
import torchxrayvision as xrv

In [50]:
#CSV Paths
metadata_path = "/ssd2/jpmokc/datasets/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv.gz"
cheXpert_path = "/ssd2/jpmokc/datasets/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv.gz"
negbio_path = "/ssd2/jpmokc/datasets/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv.gz"

d_mimic = xrv.datasets.MIMIC_Dataset(imgpath="/", csvpath=cheXpert_path, metacsvpath=metadata_path, views=['PA','AP'])


In [2]:
d_pad = xrv.datasets.PC_Dataset(imgpath="/ssd2/jpmokc/datasets/PADCHEST_SJ/image_zips", views=["PA","AP"])

In [44]:
d_mimic.csv["ProcedureCodeSequence_CodeMeaning"].unique()

array(['CHEST (PA AND LAT)', 'CHEST (PORTABLE AP)',
       'DX CHEST PORT LINE/TUBE PLCMT 1 EXAM', 'postero-anterior',
       'DX CHEST & RIBS', 'DX CHEST PORTABLE PICC LINE PLACEMENT',
       'CHEST PORT LINE PLACEMENT', 'antero-posterior',
       'TRAUMA No.2 (AP CXR & PELVIS PORT)',
       'DX CHEST PORT LINE/TUBE PLCMT 2 EXAMS', 'DX CHEST WITH DECUB',
       'CHEST (SINGLE VIEW)', 'CHEST PORT LINE/TUBE PLCT 1 EXAM',
       'DX CHEST PORT LINE/TUBE PLCMT 3 EXAMS',
       'TRAUMA #3 (PORT CHEST ONLY)',
       'DX CHEST 2 VIEW PICC LINE PLACEMENT',
       'DX CHEST PORT LINE/TUBE PLCMT 4 EXAMS', 'CHEST PRE-OP',
       'ABDOMEN (SUPINE ONLY)', 'CHEST SGL VIEW/LINE PLACEMENT',
       'CHEST (PRE-OP PA & LAT)', 'DX TRAUMA SERIES (PORTABLE)'],
      dtype=object)

##Code Sequence Meanings:

    - Chest (PA and LAT) -> Includes PA view and Lat view
    - Chest (Portable AP) -> Just a single AP view -> 19,000
    - Chest (Single view) -> Mostly AP a few PAs - Very few
    - Antero-posterior -> AP and Lat views
    - Posterior-Antero -> PA and Lat views
    
    

In [47]:
#Used this brick to apply different filters to get a gist of what views corresponded with what codesequences 

df_imageInfo = pd.concat([d_mimic.csv["study_id"],d_mimic.csv["patientid"],d_mimic.csv["view"],d_mimic.csv["ProcedureCodeSequence_CodeMeaning"]], axis=1)
filter = df_imageInfo["ProcedureCodeSequence_CodeMeaning"] == 'CHEST (PA AND LAT)'
filter2 = df_imageInfo['view'] == "AP"
df_imageInfo[filter & filter2]

,study_id,patientid,view,ProcedureCodeSequence_CodeMeaning
1,57375967,10000764,AP,CHEST (PA AND LAT)
6,53957785,10001122,AP,CHEST (PA AND LAT)
13,52823782,10002131,AP,CHEST (PA AND LAT)
19,52212843,10002559,AP,CHEST (PA AND LAT)
21,55885481,10002930,AP,CHEST (PA AND LAT)
...,...,...,...,...
63904,53069774,19994505,AP,CHEST (PA AND LAT)
63906,53860483,19994588,AP,CHEST (PA AND LAT)
63907,51399704,19994600,AP,CHEST (PA AND LAT)
63920,57654063,19996061,AP,CHEST (PA AND LAT)


In [54]:
#Different Labels for the same view
d_pad.csv['ViewPosition_DICOM'].unique()

array(['PA', nan, 'POSTEROANTERIOR', 'ANTEROPOSTERIOR', 'AP'],
      dtype=object)

In [58]:
pad_image_info = pd.concat([d_pad.csv['ImageID'],d_pad.csv['ImageDir'],d_pad.csv['view']], axis=1)

In [60]:
filter1 = pad_image_info['view'] == 'AP'
pad_image_info[filter1]

,ImageID,ImageDir,view
12,216840111366964012819207061112010293090120822_...,34,AP
38,185901399126852050366705436556047034618_8dtky4...,54,AP
83,216840111366964012558082906712010081145319302_...,33,AP
100,216840111366964012810946289282010223092157433_...,12,AP
103,216840111366964012373310883942009168083631183_...,37,AP
...,...,...,...
63552,216840111366964012373310883942009195111853512_...,37,AP
63575,216840111366964013686042548532013177085237836_...,15,AP
63689,216840111366964013439197051132012220102219417_...,19,AP
63731,216840111366964012810946289282010218091856402_...,12,AP


In [ ]:
def resize_img(path, size):
    img = skimage.io.imread(path)
    img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
    img = img.mean(2)[None, ...] # Make single color channel

    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(size)])
    img = transform(img)
    img = torch.from_numpy(img)

    return img